In [0]:
#imports
from pyspark.sql.functions import col
import pandas as pd

In [0]:
# Define nome do Shchema
schema = "agroclima.gold" 

#Retorna Tabelas do Shchema
tabelas = spark.sql(f"SHOW TABLES IN {schema}")

In [0]:
#filtra tabelas persistidas no databse e seleciona nome da tabela
tabelas = tabelas.filter(
                    col("isTemporary") == "false"
).select("tableName")

In [0]:
#função que retorna a quantidade de linhas de uma tabela
def qtd_linhas(tabela):
    df = spark.read.table(f"{schema}.{tabela}")
    return df.count()




In [0]:
# Função que retorna media de valores nulos
def media_nulos(tabela):
    df = spark.read.table("agroclima.gold.ft_temperaturas")
    percentuais_nulos = df.toPandas().isnull().sum() / df.count()
    media = percentuais_nulos.mean()
    return media


In [0]:
#Cria Lista com dicionarios de validações
validacoes = []
for tabela in tabelas.collect():
    validacoes.append({
        "FONTE_DE_DADOS": tabela["tableName"],
        "QTD_ROWS": qtd_linhas(tabela["tableName"]),
        "NULL_RATIO_MED": media_nulos(tabela["tableName"])
    })

In [0]:
# Transforma lista em Dataframe
df_valida = pd.DataFrame(validacoes)

In [0]:
display(df_valida)